In [1]:
#pip install prophet

In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import multiprocessing
from functools import partial
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from scipy import stats

In [4]:
data = pd.read_excel('data v2.xlsx')

In [5]:
df = data.iloc[:, 1:]

df.head()

,wk 1,wk 2,wk 3,wk 4,wk 5,wk 6,wk 7,wk 8,wk 9,wk 10,...,wk 53,wk 54,wk 55,wk 56,wk 57,wk 58,wk 59,wk 60,wk 61,wk 62
0,72848.0,57156.0,1099.0,71916.0,31700.0,65996.0,58028.0,60564.0,52239.0,24323.0,...,39966.0,41614.0,38862.0,14572.0,15601.0,8917.0,9358.0,24340.0,21677.0,19287.0
1,8067.0,9288.0,4099.0,8258.0,5709.0,10616.0,5009.0,6197.0,7866.0,9278.0,...,13640.0,11163.0,14303.0,8431.0,8099.0,4668.0,2701.0,4380.0,6482.0,8623.0
2,2914.0,7470.0,NaN,1202.0,12490.0,12119.0,14724.0,6245.0,1093.0,12820.0,...,5944.0,4720.0,4810.0,2805.0,2037.0,1754.0,2054.0,3074.0,4556.0,6626.0
3,138572.0,198611.0,96669.0,22963.0,104178.0,125459.0,132591.0,100540.0,161350.0,218670.0,...,116880.0,107120.0,93911.0,100258.0,1952778.0,1033274.0,1310199.0,1478926.0,2129291.0,1893435.0
4,9244.0,11096.0,9232.0,8208.0,7600.0,6944.0,4563.0,9776.0,8700.0,11534.0,...,4989.0,4932.0,5125.0,5017.0,33613.0,15980.0,20172.0,27206.0,33821.0,24743.0


In [6]:
def has_nulls(dataset):
    print("Dataset has nulls is ", dataset.isnull().values.any())

has_nulls(df)

Dataset has nulls is  True


In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)
imputer.fit(df)
df = pd.DataFrame(imputer.transform(df))

has_nulls(df)

Dataset has nulls is  False


In [8]:
def remove_outliers_zscore(sales_data, z_threshold=3):
    z_scores = np.abs(stats.zscore(sales_data))
    sales_data_no_outliers = sales_data[(z_scores < z_threshold).all(axis=1)]
    return sales_data_no_outliers

In [9]:
sales_data = df

sales_data_no_outliers = remove_outliers_zscore(sales_data)
print(sales_data_no_outliers.shape)

(15434, 62)


In [10]:
# To Convert annual
weight_column1 = 0.7
weight_column2 = 0.3

for i in range(0, 10):
    sales_data_no_outliers[i] = sales_data_no_outliers[i] * weight_column1 + sales_data_no_outliers[i+52] * weight_column2

In [11]:
sales_data_no_outliers.head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,62983.4,52493.4,12427.9,54712.8,26870.3,48872.3,43427.0,49696.8,43070.4,22812.2,...,39966.0,41614.0,38862.0,14572.0,15601.0,8917.0,9358.0,24340.0,21677.0,19287.0
1,9738.9,9850.5,7160.2,8309.9,6426.0,8831.6,4316.6,5651.9,7450.8,9081.5,...,13640.0,11163.0,14303.0,8431.0,8099.0,4668.0,2701.0,4380.0,6482.0,8623.0
2,3823.0,6645.0,3158.0,1682.9,9354.1,9009.5,10923.0,5293.7,2131.9,10961.8,...,5944.0,4720.0,4810.0,2805.0,2037.0,1754.0,2054.0,3074.0,4556.0,6626.0
4,7967.5,9246.8,7999.9,7250.7,15403.9,9654.8,9245.7,15005.0,16236.3,15496.7,...,4989.0,4932.0,5125.0,5017.0,33613.0,15980.0,20172.0,27206.0,33821.0,24743.0
5,39479.0,55438.6,39854.3,35344.0,71615.6,56102.9,54335.0,45431.5,41073.8,41185.7,...,35244.0,30758.0,31273.0,27630.0,152460.0,95613.0,103174.0,79931.0,73285.0,82744.0


In [12]:
annual_data = sales_data_no_outliers.drop(columns=range(52,62))

In [13]:
annual_data.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,62983.4,52493.4,12427.9,54712.8,26870.3,48872.3,43427.0,49696.8,43070.4,22812.2,...,65689.0,27310.0,28338.0,26669.0,43789.0,24282.0,53120.0,40401.0,31224.0,57982.0
1,9738.9,9850.5,7160.2,8309.9,6426.0,8831.6,4316.6,5651.9,7450.8,9081.5,...,22970.0,11753.0,18398.0,14626.0,16863.0,17231.0,15723.0,13696.0,12411.0,16156.0
2,3823.0,6645.0,3158.0,1682.9,9354.1,9009.5,10923.0,5293.7,2131.9,10961.8,...,4708.0,4674.0,4548.0,4386.0,4743.0,5379.0,6069.0,5402.0,6422.0,6444.0
4,7967.5,9246.8,7999.9,7250.7,15403.9,9654.8,9245.7,15005.0,16236.3,15496.7,...,8776.0,5497.0,7344.0,6591.0,6244.0,6495.0,6428.0,4725.0,4626.0,6037.0
5,39479.0,55438.6,39854.3,35344.0,71615.6,56102.9,54335.0,45431.5,41073.8,41185.7,...,45469.0,14360.0,24682.0,27983.0,29493.0,31042.0,32299.0,32459.0,30806.0,38095.0


In [14]:
annual_data.shape

(15434, 52)

In [15]:
scaler = MinMaxScaler()
normalized_sales_data = scaler.fit_transform(annual_data.T).T

# Convert the normalized array back to DataFrame
normalized_sales_data_df = pd.DataFrame(normalized_sales_data, index=annual_data.index, columns=annual_data.columns)

In [16]:
normalized_sales_data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.957762,0.793999,0.168523,0.828647,0.393988,0.737469,0.652460,0.750340,0.646893,0.330636,...,1.000000,0.400852,0.416901,0.390846,0.658112,0.353581,0.803781,0.605220,0.461955,0.879683
1,0.277461,0.281494,0.184273,0.225820,0.157741,0.244673,0.081512,0.129767,0.194775,0.253704,...,0.755601,0.350246,0.590380,0.454069,0.534909,0.548208,0.493712,0.420461,0.374024,0.509360
2,0.233801,0.467933,0.178628,0.056243,0.692699,0.664109,0.822866,0.355820,0.093495,0.826085,...,0.307226,0.304406,0.293952,0.280511,0.310130,0.362897,0.420144,0.364805,0.449432,0.451257
4,0.268060,0.345011,0.270009,0.224944,0.715362,0.369552,0.344944,0.691368,0.765432,0.720944,...,0.316692,0.119459,0.230556,0.185263,0.164391,0.179489,0.175459,0.073023,0.067068,0.151940
5,0.332631,0.543972,0.337601,0.277875,0.758192,0.552769,0.529358,0.411456,0.353750,0.355232,...,0.411952,0.000000,0.136686,0.180399,0.200395,0.220907,0.237552,0.239671,0.217782,0.314304


In [17]:
sales_data_normalized = normalized_sales_data_df

In [18]:
num_clusters = 3  
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
sales_data_normalized['cluster'] = kmeans.fit_predict(sales_data_normalized)

In [19]:
def assign_trend_label(cluster_data):
    if cluster_data['cluster'].mean() > 0.5:
        return "Emerging"
    elif cluster_data['cluster'].mean() < -0.5:
        return "Declining"
    else:
        return "Saturated"

In [20]:
cluster_trends_df = sales_data_normalized.groupby('cluster').apply(assign_trend_label)
cluster_trends_df = cluster_trends_df.reset_index()
cluster_trends_df.columns = ['cluster', 'trend']

print(cluster_trends_df)

   cluster      trend
0        0  Saturated
1        1   Emerging
2        2   Emerging


In [28]:
sales_data_normalized['cluster'].value_counts()

cluster
1    6256
2    5464
0    3714
Name: count, dtype: int64

In [69]:
def report(sales_data,search_terms):
    cluster1 = pd.DataFrame(columns=["search_terms"])
    cluster2 = pd.DataFrame(columns=["search_terms"])
    cluster3 = pd.DataFrame(columns=["search_terms","annual_data","max_value_idx","min_value_idx"])
    for i in sales_data.index:
        if(sales_data[i] == 0):
            new_row = {"search_terms":search_terms[i],"annual_data":[i for i in annual_data.iloc[i,:]]}
            #print(new_row)
            cluster1.loc[len(cluster1)] = new_row
#         elif(sales_data[i] == 1):
#             cluster2 = cluster2.append({"search_terms":search_terms[i],"annual_data":[i for i in annual_data[i]]}, ignore_index=True)
#         else:
#             cluster3 = cluster3.append({"search_terms":search_terms[i],"annual_data":[i for i in annual_data[i]]}, ignore_index=True)
    print(cluster1.shape)        
        

In [70]:
report(sales_data_normalized['cluster'],data.iloc[:,0])

IndexError: single positional indexer is out-of-bounds